# Some clean

In [5]:
import pandas as pd

# Load the Excel file
df = pd.read_excel(r"F:\PhD\RA\Schafer\IRA\data\unzipped\merge\merged4_non_missing.xlsx")

# Drop rows where DrugName is missing
df = df.dropna(subset=['DrugName'])

# Remove time from ApplicationDocsDate and keep only the date
df['ApplicationDocsDate'] = pd.to_datetime(df['ApplicationDocsDate']).dt.date

# Create 'year' column
df['year'] = pd.to_datetime(df['ApplicationDocsDate']).dt.year

# Define your unique ID columns (adjust as needed)
id_cols = ['ApplNo', 'ProductNo']

# Get the earliest year for each unique ID
df['appear'] = df.groupby(id_cols)['year'].transform('min')

# Sort for grouping
df = df.sort_values(id_cols + ['ApplicationDocsDate'])

# Create first_orig column robustly
def first_is_orig(series):
    return 1 if series.iloc[0] == 'ORIG' else 0

df['first_orig'] = df.groupby(id_cols)['SubmissionType'].transform(first_is_orig)

# **Drop records where appear < 2000**
df = df[df['appear'] >= 2000]

# Save to new Excel file
df.to_excel(r"F:\PhD\RA\Schafer\IRA\data\unzipped\merge\after00.xlsx", index=False)

In [1]:
import pandas as pd
import pdfplumber
import requests
import tempfile
import re
import time
from tqdm import tqdm

main_bullets = ["•", "\uf0b7", "·", "\u00b7", "."]

def is_two_column(page, center_width_fraction=0.02, min_text_length=26):
    width = page.width
    height = page.height
    center_start = width * (0.5 - center_width_fraction / 2)
    center_end = width * (0.5 + center_width_fraction / 2)
    center_text = page.within_bbox((center_start, 0, center_end, height)).extract_text() or ""
    return len(center_text) <= min_text_length

def extract_all_text(pdf_path):
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            two_col = is_two_column(page)
            if two_col:
                width = page.width
                left_col = page.within_bbox((0, 0, width / 2, page.height)).extract_text() or ""
                right_col = page.within_bbox((width / 2, 0, width, page.height)).extract_text() or ""
                all_text += left_col + "\n" + right_col + "\n"
            else:
                page_text = page.extract_text() or ""
                all_text += page_text + "\n"
    return all_text

def find_loose_and_dashed_section_index(lines, pattern):
    pat = re.compile(pattern, re.IGNORECASE)
    for idx, line in enumerate(lines):
        if pat.search(line.replace(" ", "")) and (
            "--" in line or "__" in line
        ):
            return idx
    return None

def find_next_dashed_section_index(lines, pattern, after_idx):
    pat = re.compile(pattern, re.IGNORECASE)
    for i in range(after_idx + 1, len(lines)):
        if pat.search(lines[i].replace(" ", "")) and (
            "--" in lines[i] or "__" in lines[i]
        ):
            return i
    return None

def find_section_indices(lines, is_2col):
    if is_2col:
        ind_start = [
            i for i, line in enumerate(lines)
            if re.search(r"indicat.{0,30}usag", line.replace(" ", "").lower())
        ]
        dos_start = [
            i for i, line in enumerate(lines)
            if re.search(r"dosag.{0,20}admin", line.replace(" ", "").lower())
        ]
        contra_start = [
            i for i, line in enumerate(lines)
            if re.search(r"contraindicat", line.replace(" ", "").lower())
        ]
    else:
        ind_start = [
            i for i, line in enumerate(lines)
            if re.match(r"^\s*(\d+[\.\)]\s*)?[-—–\s]*indicat.{0,30}usag", line.replace(" ", "").lower())
        ]
        dos_start = [
            i for i, line in enumerate(lines)
            if re.match(r"^\s*(\d+[\.\)]\s*)?[-—–\s]*dosag.{0,20}admin", line.replace(" ", "").lower())
        ]
        contra_start = [
            i for i, line in enumerate(lines)
            if re.match(r"^\s*(\d+[\.\)]\s*)?[-—–\s]*contraindicat", line.replace(" ", "").lower())
        ]
    return ind_start, dos_start, contra_start

def get_section_lines_with_gap(ind_idx, dos_indices, lines, gaps=[5, 10, 15]):
    for gap in gaps:
        valid_dos_indices = [i for i in dos_indices if i - ind_idx > gap]
        if valid_dos_indices:
            first_valid = valid_dos_indices[0]
            return lines[ind_idx+1:first_valid], gap
    return [], None

def extract_section_loose(lines, is_2col):
    if is_2col:
        ind_pattern = r"indicat.{0,30}usag"
        dos_pattern = r"dosag.{0,20}admin"
        # 1. Try dashed/underscored header logic
        start_idx = find_loose_and_dashed_section_index(lines, ind_pattern)
        end_idx = None
        if start_idx is not None:
            end_idx = find_next_dashed_section_index(lines, dos_pattern, start_idx)
        if start_idx is not None and end_idx is not None:
            return "\n".join(lines[start_idx + 1 : end_idx])
        # 2. Fallback: original gap logic with loose detection
        ind_start, dos_start, _ = find_section_indices(lines, is_2col)
        if ind_start and dos_start:
            section_lines, used_gap = get_section_lines_with_gap(ind_start[0], dos_start, lines)
            if section_lines:
                return "\n".join([lines[ind_start[0]]] + section_lines)
            else:
                return "\n".join(lines[ind_start[0]:])
        elif ind_start:
            return "\n".join(lines[ind_start[0]:])
        else:
            return ""
    else:
        ind_start, _, contra_start = find_section_indices(lines, is_2col)
        if ind_start and contra_start:
            stop = [i for i in contra_start if i > ind_start[0]]
            if stop:
                stop = stop[0]
                return "\n".join(lines[ind_start[0]:stop])
        elif ind_start:
            return "\n".join(lines[ind_start[0]:])
        else:
            return ""
    return ""

def split_by_main_bullet(text):
    bullet_chars = ''.join(re.escape(b) for b in main_bullets if b != ".")
    pattern = r'((?:^|\n)[{}]\s)'.format(bullet_chars)
    pattern += r'|((?:^|\n)\.\s)'
    splits = [m.start() for m in re.finditer(pattern, text)]
    if not splits:
        return [text]
    sections = []
    for i, idx in enumerate(splits):
        end = splits[i+1] if i+1 < len(splits) else len(text)
        sections.append(text[idx:end].strip())
    return sections

def is_sub_bullet_line(line):
    line_stripped = line.lstrip()
    return line_stripped.startswith("o ") or line_stripped.startswith("\u25cb")

def count_sub_bullets(section):
    lines = section.splitlines()
    count = 0
    for line in lines:
        if is_sub_bullet_line(line):
            count += 1
    return count if count > 0 else 1

def should_exclude_section(section):
    return 'limitation of use' in section.lower()

def count_indicat_and_treatment(section_text):
    lines = section_text.splitlines()
    count_indicat = max(0, sum(1 for line in lines if re.search(r"indicated", line, re.IGNORECASE)))
    count_treatment = sum(1 for line in lines if re.search(r"treatm", line, re.IGNORECASE))
    return count_indicat, count_treatment

def find_max_numbered_bullet(lines):
    numbers = []
    for line in lines:
        match = re.match(r"^\s*(\d+)[\.\)]", line)
        if match:
            numbers.append(int(match.group(1)))
    return max(numbers) if numbers else 0

def count_indications_keyword_and_number(section_text):
    lines = section_text.splitlines()
    count_indicat, count_treatment = count_indicat_and_treatment(section_text)
    max_keyword = max(count_indicat, count_treatment)
    max_number = find_max_numbered_bullet(lines)
    final_count = max(max_keyword, max_number)
    return final_count

def get_indication_count(pdf_url):
    try:
        response = requests.get(pdf_url, headers={"User-Agent": "Mozilla/5.0"}, timeout=60)
        response.raise_for_status()
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(response.content)
            tmp_path = tmp_file.name
        with pdfplumber.open(tmp_path) as pdf:
            first_page = pdf.pages[0]
            is_2col = is_two_column(first_page)
            all_text = extract_all_text(tmp_path)
            lines = all_text.splitlines()
            section = extract_section_loose(lines, is_2col=is_2col)
            if not section.strip():
                return None, False
            if is_2col:
                main_sections = [
                    s for s in split_by_main_bullet(section)
                    if s.strip() and not should_exclude_section(s)
                ]
                count = 0
                for sec in main_sections:
                    cnt = count_sub_bullets(sec)
                    count += cnt
            else:
                count = count_indications_keyword_and_number(section)
            return count, True
    except Exception as e:
        print(f"[ERROR] PDF extraction failed: {e}")
        return None, False

def run_auto_pdf_extraction(
    excel_path,
    out_path="indication_results.xlsx",
    url_col="ApplicationDocsURL",
    max_retries=3
):
    df = pd.read_excel(excel_path)
    # --- Apply your mask ---
    mask = (
        (df["ApplicationDocsTypeID"] == 2) &
        (df["ApplicationDocsURL"].notnull()) &
        (
            (df["ActionTypes_LookupID"] < 13) |
            (df["SubmissionType"] == "ORIG")
        )
    )
    df_proc = df.loc[mask].copy()
    indication_counts = [None] * len(df_proc)
    open_successes = [False] * len(df_proc)
    failed_indices = list(range(len(df_proc)))

    for attempt in range(max_retries):
        print(f"\n--- Iteration {attempt+1} ---")
        still_failed = []
        for idx in tqdm(failed_indices):
            row = df_proc.iloc[idx]
            count, opened = get_indication_count(row[url_col])
            indication_counts[idx] = count
            open_successes[idx] = opened
            if not opened:
                still_failed.append(idx)
                time.sleep(1)
        failed_indices = still_failed
        print(f"Iteration {attempt+1}: Failed PDFs = {len(failed_indices)}")
        if not failed_indices:
            break

    df_proc["indication_count"] = indication_counts
    df_proc["pdf_opened"] = open_successes
    df_proc.to_excel(out_path, index=False)
    print(f"Results saved to {out_path}")
    if failed_indices:
        print(f"Still failed indices after {max_retries} iterations: {failed_indices}")


In [6]:
# ---- Usage Example ----
run_auto_pdf_extraction(
    excel_path="F:/PhD/RA/Schafer/IRA/data/unzipped/merge/main/exclude_part_1.xlsx",    
    out_path="F:/PhD/RA/Schafer/IRA/data/unzipped/merge/processed/exclude_part_1_processed.xlsx"
)


--- Iteration 1 ---


 40%|████      | 153/381 [08:31<15:42,  4.13s/it]

[ERROR] PDF extraction failed: 404 Client Error: Not Found for url: https://www.accessdata.fda.gov/drugsatfda_docs/nda/2002/20-839s019_ClopidogrelPlavix.htm


 41%|████      | 156/381 [08:39<11:36,  3.09s/it]

[ERROR] PDF extraction failed: 404 Client Error: Not Found for url: https://www.accessdata.fda.gov/drugsatfda_docs/nda/2002/20-839s019_ClopidogrelPlavix.htm


...

[ERROR] PDF extraction failed: 404 Client Error: Not Found for url: https://www.accessdata.fda.gov/drugsatfda_docs/nda/2002/20-839s019_ClopidogrelPlavix.htm


 93%|█████████▎| 65/70 [02:46<00:13,  2.79s/it]

[ERROR] PDF extraction failed: Bounding box (583.3695, 0, 607.1804999999999, 841.88998) is not fully within parent page bounding box (0, 0.0009800000000268483, 1190.55, 841.8909600000001)


100%|██████████| 70/70 [03:07<00:00,  2.67s/it]


Iteration 3: Failed PDFs = 70
Results saved to F:/PhD/RA/Schafer/IRA/data/unzipped/merge/processed/exclude_part_1_processed.xlsx
Still failed indices after 3 iterations: [13, 16, 19, 22, 50, 52, 56, 59, 60, 65, 66, 71, 72, 77, 78, 83, 84, 89, 90, 97, 100, 101, 102, 114, 153, 156, 164, 165, 166, 167, 168, 169, 176, 179, 180, 187, 188, 195, 196, 203, 204, 211, 212, 218, 220, 263, 264, 265, 267, 268, 269, 271, 272, 273, 275, 276, 277, 279, 280, 281, 300, 345, 350, 352, 357, 365, 373, 375, 377, 379]


In [7]:
# ---- Usage Example ----
run_auto_pdf_extraction(
    excel_path="F:/PhD/RA/Schafer/IRA/data/unzipped/merge/main/exclude_part_2.xlsx",    
    out_path="F:/PhD/RA/Schafer/IRA/data/unzipped/merge/processed/exclude_part_2_processed.xlsx"
)


--- Iteration 1 ---


 10%|▉         | 52/523 [01:45<16:14,  2.07s/it]Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
 11%|█         | 57/523 [02:06<33:03,  4.26s/it]Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke c

Iteration 1: Failed PDFs = 63

--- Iteration 2 ---


100%|██████████| 63/63 [03:00<00:00,  2.87s/it]


Iteration 2: Failed PDFs = 63

--- Iteration 3 ---


100%|██████████| 63/63 [02:59<00:00,  2.85s/it]


Iteration 3: Failed PDFs = 63
Results saved to F:/PhD/RA/Schafer/IRA/data/unzipped/merge/processed/exclude_part_2_processed.xlsx
Still failed indices after 3 iterations: [1, 2, 7, 8, 13, 14, 19, 20, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 68, 75, 82, 99, 115, 131, 147, 164, 168, 172, 176, 241, 249, 257, 265, 266, 267, 268, 269, 270, 394, 403, 418, 419, 420, 422, 423, 424, 448, 449, 451, 456, 461, 466, 471, 475, 522]


In [8]:
# ---- Usage Example ----
run_auto_pdf_extraction(
    excel_path="F:/PhD/RA/Schafer/IRA/data/unzipped/merge/main/exclude_part_3.xlsx",    
    out_path="F:/PhD/RA/Schafer/IRA/data/unzipped/merge/processed/exclude_part_3_processed.xlsx"
)


--- Iteration 1 ---


 11%|█         | 64/600 [03:42<42:37,  4.77s/it]

[ERROR] PDF extraction failed: Bounding box (599.76049, 0, 624.24051, 792.0) is not fully within parent page bounding box (-184.991, 98.5, 1039.01, 890.5)


 11%|█         | 66/600 [03:48<34:59,  3.93s/it]

[ERROR] PDF extraction failed: Bounding box (599.76049, 0, 624.24051, 792.0) is not fully within parent page bounding box (-184.991, 98.5, 1039.01, 890.5)


...

[ERROR] PDF extraction failed: Bounding box (599.76049, 0, 624.24051, 792.0) is not fully within parent page bounding box (-184.991, 98.5, 1039.01, 890.5)


 71%|███████   | 27/38 [01:32<00:32,  2.98s/it]

[ERROR] PDF extraction failed: Bounding box (599.76049, 0, 624.24051, 792.0) is not fully within parent page bounding box (-184.991, 98.5, 1039.01, 890.5)


100%|██████████| 38/38 [02:29<00:00,  3.94s/it]


Iteration 3: Failed PDFs = 38
Results saved to F:/PhD/RA/Schafer/IRA/data/unzipped/merge/processed/exclude_part_3_processed.xlsx
Still failed indices after 3 iterations: [4, 9, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 51, 52, 53, 59, 64, 66, 74, 75, 151, 153, 162, 171, 200, 249, 332, 337]


In [9]:
# ---- Usage Example ----
run_auto_pdf_extraction(
    excel_path="F:/PhD/RA/Schafer/IRA/data/unzipped/merge/main/exclude_part_4.xlsx",    
    out_path="F:/PhD/RA/Schafer/IRA/data/unzipped/merge/processed/exclude_part_4_processed.xlsx"
)


--- Iteration 1 ---


 96%|█████████▌| 493/513 [27:18<00:37,  1.87s/it]Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
 96%|█████████▋| 494/513 [27:21<00:40,  2.15s/it]Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
 96%|█████████▋| 495/513 [27:24<00:45,  2.50s/it]Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is 

Iteration 1: Failed PDFs = 11

--- Iteration 2 ---


100%|██████████| 11/11 [00:28<00:00,  2.59s/it]


Iteration 2: Failed PDFs = 11

--- Iteration 3 ---


100%|██████████| 11/11 [00:27<00:00,  2.50s/it]


Iteration 3: Failed PDFs = 11
Results saved to F:/PhD/RA/Schafer/IRA/data/unzipped/merge/processed/exclude_part_4_processed.xlsx
Still failed indices after 3 iterations: [9, 10, 48, 49, 174, 338, 344, 350, 356, 362, 460]


In [10]:
# ---- Usage Example ----
run_auto_pdf_extraction(
    excel_path="F:/PhD/RA/Schafer/IRA/data/unzipped/merge/main/exclude_part_5.xlsx",    
    out_path="F:/PhD/RA/Schafer/IRA/data/unzipped/merge/processed/exclude_part_5_processed.xlsx"
)


--- Iteration 1 ---


  2%|▏         | 18/748 [01:08<49:05,  4.03s/it] Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
  3%|▎         | 19/748 [01:11<46:51,  3.86s/it]Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke 

[ERROR] PDF extraction failed: 404 Client Error: Not Found for url: https://www.accessdata.fda.gov/drugsatfda_docs/label/2018/103000s5308lbl.pdf


 21%|██        | 157/748 [08:35<52:23,  5.32s/it]  

[ERROR] PDF extraction failed: 404 Client Error: Not Found for url: https://www.accessdata.fda.gov/drugsatfda_docs/label/2018/103000s5308lbl.pdf


...

[ERROR] PDF extraction failed: No /Root object! - Is this really a PDF?


 98%|█████████▊| 94/96 [03:06<00:05,  2.52s/it]

[ERROR] PDF extraction failed: 404 Client Error: Not Found for url: https://www.accessdata.fda.gov/drugsatfda_docs/label/2018/125554s061lbl.pdf


100%|██████████| 96/96 [03:10<00:00,  1.98s/it]


Iteration 3: Failed PDFs = 96
Results saved to F:/PhD/RA/Schafer/IRA/data/unzipped/merge/processed/exclude_part_5_processed.xlsx
Still failed indices after 3 iterations: [3, 75, 78, 83, 88, 93, 98, 103, 108, 113, 122, 123, 124, 127, 136, 144, 145, 148, 157, 165, 166, 169, 178, 186, 187, 190, 199, 207, 208, 211, 220, 228, 229, 235, 236, 242, 243, 249, 250, 256, 257, 263, 264, 270, 271, 277, 278, 284, 285, 291, 292, 298, 299, 305, 308, 312, 315, 319, 322, 326, 329, 333, 336, 340, 343, 347, 350, 354, 357, 361, 364, 368, 371, 375, 376, 377, 378, 385, 386, 387, 388, 427, 428, 429, 431, 432, 442, 453, 455, 456, 466, 615, 621, 622, 646, 746]


In [11]:
# ---- Usage Example ----
run_auto_pdf_extraction(
    excel_path="F:/PhD/RA/Schafer/IRA/data/unzipped/merge/main/exclude_part_6.xlsx",    
    out_path="F:/PhD/RA/Schafer/IRA/data/unzipped/merge/processed/exclude_part_6_processed.xlsx"
)


--- Iteration 1 ---


  6%|▌         | 42/745 [02:33<52:59,  4.52s/it] 

[ERROR] PDF extraction failed: Bounding box (291.68475, 0, 303.59024999999997, 841.8893999999999) is not fully within parent page bounding box (0.0, 14.173199999999952, 595.275, 856.0625999999999)


  6%|▌         | 43/745 [02:39<58:43,  5.02s/it]

[ERROR] PDF extraction failed: Bounding box (291.68475, 0, 303.59024999999997, 841.8893999999999) is not fully within parent page bounding box (0.0, 14.173199999999952, 595.275, 856.0625999999999)


...

[ERROR] PDF extraction failed: 404 Client Error: Not Found for url: https://www.accessdata.fda.gov/drugsatfda_docs/label/2017/761054Orig1s000TOC.cfm


 97%|█████████▋| 33/34 [02:14<00:03,  3.70s/it]

[ERROR] PDF extraction failed: Bounding box (291.68523999999996, 0, 303.59076, 913.89) is not fully within parent page bounding box (0.0, 36.0, 595.276, 949.89)


100%|██████████| 34/34 [02:17<00:00,  4.05s/it]


Iteration 3: Failed PDFs = 34
Results saved to F:/PhD/RA/Schafer/IRA/data/unzipped/merge/processed/exclude_part_6_processed.xlsx
Still failed indices after 3 iterations: [42, 43, 105, 121, 125, 129, 133, 137, 141, 167, 172, 177, 272, 277, 365, 378, 416, 420, 424, 430, 511, 527, 529, 539, 542, 563, 564, 567, 568, 571, 572, 714, 724, 728]


In [5]:
import pandas as pd
import math

# Load the full Excel file
input_path = "F:/PhD/RA/Schafer/IRA/data/unzipped/merge/excluded_after00_updated.xlsx"
df = pd.read_excel(input_path)

n_splits = 6
rows_per_split = math.ceil(len(df) / n_splits)

for i in range(n_splits):
    start = i * rows_per_split
    end = start + rows_per_split
    df_part = df.iloc[start:end]
    out_path = f"F:/PhD/RA/Schafer/IRA/data/unzipped/merge/main/exclude_part_{i+1}.xlsx"
    df_part.to_excel(out_path, index=False)
    print(f"Saved file {i+1}: {out_path} ({len(df_part)} rows)")

Saved file 1: F:/PhD/RA/Schafer/IRA/data/unzipped/merge/main/exclude_part_1.xlsx (5875 rows)
Saved file 2: F:/PhD/RA/Schafer/IRA/data/unzipped/merge/main/exclude_part_2.xlsx (5875 rows)
Saved file 3: F:/PhD/RA/Schafer/IRA/data/unzipped/merge/main/exclude_part_3.xlsx (5875 rows)
Saved file 4: F:/PhD/RA/Schafer/IRA/data/unzipped/merge/main/exclude_part_4.xlsx (5875 rows)
Saved file 5: F:/PhD/RA/Schafer/IRA/data/unzipped/merge/main/exclude_part_5.xlsx (5875 rows)
Saved file 6: F:/PhD/RA/Schafer/IRA/data/unzipped/merge/main/exclude_part_6.xlsx (5871 rows)


# FIX SOME

In [3]:
import pandas as pd

# Load both files
df_merged = pd.read_excel(r"F:\PhD\RA\Schafer\IRA\data\unzipped\merge\merged4_non_missing.xlsx")
df_after00 = pd.read_excel(r"F:\PhD\RA\Schafer\IRA\data\unzipped\merge\after00.xlsx")

# Define ID columns
id_cols = ['ApplNo', 'ProductNo']

# Get records that are in merged but NOT in after00
df_excluded = df_merged.merge(df_after00[id_cols], on=id_cols, how='left', indicator=True)
df_excluded = df_excluded[df_excluded['_merge'] == 'left_only'].drop(columns=['_merge'])

# Drop rows with missing DrugName
df_excluded = df_excluded.dropna(subset=['DrugName'])

# Convert date columns
df_excluded['ApplicationDocsDate'] = pd.to_datetime(df_excluded['ApplicationDocsDate'], errors='coerce')
df_excluded['SubmissionStatusDate'] = pd.to_datetime(df_excluded['SubmissionStatusDate'], errors='coerce')

# Take the maximum of both dates
df_excluded['max_date'] = df_excluded[['ApplicationDocsDate', 'SubmissionStatusDate']].max(axis=1)
df_excluded['year'] = df_excluded['max_date'].dt.year

# Calculate first year this ID appeared
df_excluded['appear'] = df_excluded.groupby(id_cols)['year'].transform('min')

# Exclude appear < 2000
df_excluded = df_excluded[df_excluded['appear'] >= 2000]

# Sort by ID and SubmissionStatusDate
df_excluded = df_excluded.sort_values(id_cols + ['SubmissionStatusDate'])

# Create first_orig column
def first_is_orig(series):
    return 1 if series.iloc[0] == 'ORIG' else 0

df_excluded['first_orig'] = df_excluded.groupby(id_cols)['SubmissionType'].transform(first_is_orig)

# Save to Excel
df_excluded.to_excel(r"F:\PhD\RA\Schafer\IRA\data\unzipped\merge\excluded_after00_updated.xlsx", index=False)
